import thư viện

In [20]:
import pandas as pd
import requests
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# ===== Hàm tiện ích lấy bảng từ Wikipedia =====
def get_wiki_tables(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return pd.read_html(response.text)


# 1. Thu thập dữ liệu từ Wikipedia

In [23]:
# ===== 1. Dữ liệu Population =====
url_pop = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population"
tables_pop = get_wiki_tables(url_pop)

# tìm bảng có cột Population
df_pop = None
for tbl in tables_pop:
    if "Population" in tbl.columns:
        df_pop = tbl
        break

df_pop.columns = ["Location", "Population", "% of world", "Date", "Source", "Notes"]
df_pop = df_pop[["Location", "Population"]].copy()

# làm sạch Population
df_pop["Population"] = (
    df_pop["Population"]
    .astype(str)
    .str.replace(",", "")
    .str.replace(r"\[.*\]", "", regex=True)
)
df_pop["Population"] = pd.to_numeric(df_pop["Population"], errors="coerce")
df_pop = df_pop[df_pop["Location"] != "World"].dropna()


C:\Users\dkmme\AppData\Local\Temp\ipykernel_13604\1751884405.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(response.text)


# 2. Làm sạch dữ liệu

In [27]:
# ===== 2. Dữ liệu Area =====
url_area = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_area"
tables_area = get_wiki_tables(url_area)

print("Có tất cả", len(tables_area), "bảng")

# Tìm bảng có cột chứa chữ "Area"
df_area = None
for i, tbl in enumerate(tables_area):
    print(f"--- Bảng {i} ---", tbl.shape)
    print(tbl.head(3), "\n")
    # kiểm tra cột nào có chữ Area hoặc km2
    if any("Area" in str(c) or "km2" in str(c) for c in tbl.columns):
        if tbl.shape[0] > 100:  # bảng lớn, nhiều dòng
            df_area = tbl
            print(f"Chọn bảng {i}")
            break

if df_area is None:
    raise ValueError("Không tìm thấy bảng Area thích hợp!")

# Đặt lại tên cột: lấy 3 cột đầu (Rank, Location, Area)
df_area = df_area.iloc[:, :3]
df_area.columns = ["Rank", "Location", "Area(km2)"]

# Chỉ giữ Location và Area
df_area = df_area[["Location", "Area(km2)"]].copy()

# Làm sạch dữ liệu số
df_area["Area(km2)"] = (
    df_area["Area(km2)"]
    .astype(str)
    .str.replace(",", "")
    .str.replace(r"\[.*\]", "", regex=True)
)
df_area["Area(km2)"] = pd.to_numeric(df_area["Area(km2)"], errors="coerce")
df_area = df_area.dropna()

print("Sau khi làm sạch:", df_area.shape)
print(df_area.head())


Có tất cả 4 bảng
--- Bảng 0 --- (1, 1)
                                                0
0  Countries of the world by area (legend in km2) 

--- Bảng 1 --- (262, 7)
  Unnamed: 0 Country / dependency         Total in km2 (mi2)  \
0          –                Earth  510,072,000 (196,940,000)   
1          1               Russia     17,098,246 (6,601,667)   
2          –           Antarctica     14,200,000 (5,480,000)   

          Land in km2 (mi2)         Water in km2 (mi2)  % water Unnamed: 6  
0  148,940,000 (57,506,000)  361,132,000 (139,434,000)     70.8        NaN  
1    16,376,870 (6,323,142)          721,380 (278,530)      4.2        [b]  
2    14,200,000 (5,480,000)                          0      0.0        [c]   

Chọn bảng 1
Sau khi làm sạch: (0, 2)
Empty DataFrame
Columns: [Location, Area(km2)]
Index: []


C:\Users\dkmme\AppData\Local\Temp\ipykernel_13604\1751884405.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(response.text)


# 3. Chuẩn hóa và PCA

In [29]:
# Ghép 2 bảng theo tên quốc gia
df_merged = pd.merge(df_pop, df_area, on="Location", how="inner")

print("Sau khi ghép:", df_merged.shape)
print(df_merged.head())


Sau khi ghép: (0, 3)
Empty DataFrame
Columns: [Location, Population, Area(km2)]
Index: []


# 4. Trực quan hóa PCA

In [33]:
df_merged["Density"] = df_merged["Population"] / df_merged["Area(km2)"]


In [ ]:
features = ["Population", "Area(km2)", "Density"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_merged[features])

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

df_merged["PC1"] = X_pca[:, 0]
df_merged["PC2"] = X_pca[:, 1]

print("Explained variance ratio:", pca.explained_variance_ratio_)
print(df_merged.head())


ValueError: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required by StandardScaler.